In [11]:
import sys, os
import pandas as pd
import numpy as np
import json
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.core.display import display
from IPython.core.display import HTML

import seaborn as sns
from scipy import stats
import matplotlib as mpl

# import tensorflow as tf

"Who needs friends? Zanzil just makes his own!"

친구가 없으면 어때, 잔질처럼 이렇게 만들면 되는데!  

WOW Zul'Gurub Zanzil

와우 줄구룹 잔질, 의술사

### 클래스 목록
- AI플레이어
- 유저
- 카드 덱

### 설정 값
- 게임 세팅
- 유저 수

### 플레이 함수
- 카드 돌리기
- 딜미스 판단
- 공약
- 기루다, 조커콜, 마이티 선언
- 주공이 프랜드 선택
- 플레이 시작하기

### 초반 구현 필요한 부분
- 룰대로 게임이 돌아가는것
- AI개발모드(플레이어 없이/있이 플레이 가능하게)

### 중반 구현 필요
- AI, 인터페이스, 그래픽, 변형룰 옵션, 누적 점수 계산

### 후반 구현
- 앱으로 만들기
- 플레이스토어 등록

### 최후반 옵션
- 리플레이 저장하면 어떨까?
    - 어케 해야 마이티 잘하는지 컴한테 배우게 ㅋㅋㅋㅋ

In [ ]:
import gamefunctions as gf
from player import Player
from user import User
from environment import Environment

import re, random

In [ ]:
# from gamefunctions import *

## 진행

In [2]:
env = Environment()
# 룰 세팅
playerNumber, rule = gf.ruleSetting()

# 덱 생성
deck = gf.deckGenerator()
# 무덤 선언
grave = []

In [3]:
### main

# 룰에 따른 플레이어 수 맞추기
if rule == "defalt":
    k = 1
elif rule == "AIs":
    k = 0
else: print('!!! Setting Error !!!')

# 플레이어 생성
players = list(range(0,playerNumber))
playerName = ['윤희', '상훈', '예찬', '광호', '길용', '루피']
# random.shuffle(playerName)
for i in range(playerNumber-k):
    players[i] = Player()
    players[i].name = playerName[i]
#     print(players[i].name) #체크포인트, cp, 사실 별 필요 없음.
if k == 1:
    players[playerNumber-1] = User() #유저(닝겐)은 특별취급~!
    players[playerNumber-1].name = playerName[playerNumber-1]
#     print(players[playerNumber-1].name)

# 포지션을 정한다...
# position = [0,1,2,3,4]
turn = list(range(0,playerNumber)) #턴(포지션으로 생각해도됨), 게임을 끝내고 새게임 시작할때 바꿀 수 있을.
random.shuffle(turn)
env.locations = turn

In [4]:
# for i in 0,1,2,3,4:
#     print(players[i].name + str(sorted(players[i].hand)))

In [5]:
# 카드 나눠주기 & 딜미스 판단
mighty, jokerCall = gf.setMightyJokercall()
joker = ['joker', 0]

missCount = playerNumber #딜미스 카운트
while missCount>0:
    grave = gf.handout(players, deck, grave)
    missCount = playerNumber
    for i in range(playerNumber):
        if gf.dealCount(players[i].hand) >=2: missCount -=1
#     if k ==1:
#         if dealCount(user.hand)>=2: missCount -=1
#     print(missCount)
    if missCount>0: print('Miss!')
print(grave)

[['joker', 0], ['S', 14], ['S', 13]]


In [27]:
# 주공 공약
biddingCount = playerNumber
turnCount = 0
others = ["",0]
preOthers = ["",11]
kingNo = 0

# for i in turn:
Jugong = 0 #주공: players[turn[Jugong]]
turnStart = 0 #매 턴 시작을 누가하는지 (첫턴엔 주공, 그다음부턴 이긴사람)
while biddingCount >0:
    others = players[turn[turnCount]].pledge(preOthers) #인자로 다른애들 공약을 넣어야 
    print("")
    adjustment = 0
    if others[0] == 'noGiru':
        adjustment = 1
    if others[1]>preOthers[1] - adjustment:
        preOthers = others
        kingNo = turn[turnCount]
        biddingCount = playerNumber-1
        
    else: biddingCount -=1
    print(players[turn[turnCount]].name+"'s bidding is: "+str(others[0]) + ", " +str(others[1]))
    
    # 원형 큐 처럼 반복되게 하는거... 나는 바보같은 방법밖에 생각안나네 ㅠ
    if turnCount < playerNumber-1:
        turnCount+= 1
    else: turnCount = 0
    
    if biddingCount == 0 :
        players[turn[turnCount]].role = 2
        turnStart = turnCount
    #순서대로 공약을 한다.
        # 공약이 안 나오면 카운트를 -1 한다
        # 공약이 나오면 카운트를 playerNumber-1 한다
        #계속 반복한다


길용's bidding is: , 0

상훈's bidding is: , 1

예찬's bidding is: noKiru, 11

윤희's bidding is: , 1

광호's bidding is: , 1

길용's bidding is: , 0

상훈's bidding is: , 1


In [28]:
# 마이티: SA, S가 기루다일 땐 DA(빨마,빨강마이티)
# 조커콜: C3, C가 기루다일 땐 S3(관악, 경기에선 H3)
giruda = preOthers[0]
env.giruda = giruda
pledge = preOthers[1]
env.bidding = pledge
players[kingNo].role = [1,0]   #00:야당, 01:프랜드, 10: 주공
env.setDeclarer(kingNo)
if giruda == "S": mighty = ['D', 14]
else: mighty = ['S', 14]
if giruda == "C": jokerCall = ['H', 3]
else: jokerCall = ['C', 3]    

In [29]:
preOthers

['noKiru', 11]

In [31]:
kingNo

2

In [33]:
turnStart = kingNo

def turnSetting(turnStart, turn): #턴스타트는 int, turn은 list
    roundTurn = turn
    while turnStart != roundTurn[0]:
        roundTurn.append(roundTurn[0])
        roundTurn.pop(0)
    return roundTurn

In [35]:
sorted(players[kingNo].hand)

[['C', 11],
 ['D', 4],
 ['D', 10],
 ['D', 12],
 ['D', 13],
 ['H', 5],
 ['H', 7],
 ['H', 11],
 ['H', 14],
 ['S', 11]]

In [36]:
grave

[['joker', 0], ['S', 14], ['S', 13]]

In [37]:
giruda

'noKiru'

In [38]:
grave = players[kingNo].kingGrave(grave, giruda)

In [39]:
print(sorted(players[kingNo].hand))
print(grave)

[['C', 11], ['D', 10], ['D', 12], ['D', 13], ['H', 11], ['H', 14], ['S', 11], ['S', 13], ['S', 14], ['joker', 0]]
[['D', 4], ['H', 5], ['H', 7]]


In [40]:
for i in turn:
    if i == kingNo: print(str(sorted(players[i].hand)) + " <-- I'm the King!!")
    else: print(sorted(players[i].hand))
    print()
print(str(grave)+' <-- This is grave')

[['C', 5], ['C', 6], ['C', 7], ['C', 9], ['D', 5], ['H', 12], ['S', 2], ['S', 4], ['S', 5], ['S', 10]]

[['C', 4], ['C', 12], ['C', 14], ['D', 6], ['H', 8], ['H', 13], ['S', 6], ['S', 7], ['S', 9], ['S', 12]]

[['C', 11], ['D', 10], ['D', 12], ['D', 13], ['H', 11], ['H', 14], ['S', 11], ['S', 13], ['S', 14], ['joker', 0]] <-- I'm the King!!

[['C', 3], ['C', 10], ['C', 13], ['D', 2], ['D', 3], ['D', 7], ['D', 14], ['H', 4], ['H', 6], ['S', 3]]

[['C', 2], ['C', 8], ['D', 8], ['D', 9], ['D', 11], ['H', 2], ['H', 3], ['H', 9], ['H', 10], ['S', 8]]

[['D', 4], ['H', 5], ['H', 7]] <-- This is grave


In [41]:
## callFriend
friendCard = players[kingNo].callFriend(giruda)
print(friendCard)
for i in turn:
    if friendCard in players[i].hand:
        players[i].role = [0,1]
#         break

# friendNo = players[kingNo].callFriend()
# players[friendNo].role = [0,1]

['noKiru', 14]


### From now on
첫 라운드 
 - 조커 힘 약함

매 라운드 
- 핸드 1 이상인동안 반복
- 조커 내도 됨

- 선카 나오는거 보고 turnShape 정하기
- 조커를 첫카드로 낼 때는 무슨 카드를 선카로 할 지 선택
- player 클래스 내에 possibleCards 함수 만들기. 그 중에서 고르도록
- pickCard 함수에서는 그 중에 랜덤하게 내기
- 라운드에 나온 카드가 뭐였는지 기록
- 라운드 끝날 때는 무덤에 라운드 카드 넣기
- 무덤이 2개가 되야할듯. 1개는 kingGrave(버린 카드 3개 더 안다 남보다), 다른건 normalGrave


In [104]:
# only king knows what cards are in initial grave.
graveKing = grave.copy()
graveNormal = grave.copy()
roundNo = 0
turnShape = ""
roundHistory
roundCard = [0,0,0,0,0]
# 누가 뭘 냈는지를 기억해야 하는데, 걍 리스트로 던져주면 어떻게 알지....
# 뭔가 방법이 필요하다.
# 매 턴 나온 카드를 53비트로 수정해서 넣어줘야 하나...?

In [ ]:
# first round
# 줄줄이 짜는게 좋냐 함수로 만들어서 콜하는게 좋냐
# 조커콜이 첫 카드면 조커콜 처리 함수 불러오자
# 일단 짜보자


"""
첫번째 사람이 카드내기: 첫턴 아무거나 낼 수 있다 - (첫턴 한정)기루 힘 없고 조커 힘 없다
    낸 카드에 대해서 turnShape 정해진다
    조커콜인지 확인
"""
roundCards[0] = players[king].pickCard(env.bits) #말고도 giruda, round, history, score, 

"""
두번째 ~ 마지막 카드내기: turnShape가 핸드에 있나/없나 에 따라서 낼 수 있는 카드가 다르다
    카드를 낸다
    고민... 자리에 대한 정보를 어떻게 처리할지
    고민... 각 턴에 나온 히스토리를 어떻게 처리할지...
"""


"""
결과 판단:
    점수 누가 먹었는지 판단 --> 스코어 더한다.
    라운드에 나온 카드는 grave에 추가한다
    다음턴을 위해 순서를...
자리에 대한 정보를 어떻게 tf에 넣어줄지도 고민이다....
"""

In [20]:
range(1,5)

range(1, 5)

In [18]:
import random

In [16]:
k = ['a','b','c','d']
a = 'a'

In [19]:
k[random.randrange(0,len(k))]

'd'

In [14]:
k.remove(a)

In [7]:
k.remove('b')

In [20]:
k

['a', 'b', 'c', 'd']

In [ ]:
# test for the first round

t = gf.roundTrun(turn, kingNo)

startCard = players[t[i]].pickCard(mighty)
if startCard[0] == 'joker':
    turnShape = players[[turn[i]]].sayShape()
else:
    turnShaper = startCard[0]
    
for i in range(1,len(t)):
    startCard = players[t[i]].pickCard(mighty)
    
    if giruda == 'noGiru'
    
    if startCard == jokerCall:

In [105]:
# second ~ last rounds
while len(players[0].hand) > 0:
    # 첫 플레이어는 아무거나 낼 수 있다
    ## turnShape를 선언해야 한다
    # 두번째 플레이어부터는 
    
    break

In [ ]:
# test for each rounds

In [106]:
bits = ['0'] *53


In [108]:
len(bits)

53

## 함수 연습 및 메모

In [13]:
print("{} and {}".format("a", 'b'))

a and b


In [469]:
#핸드에서 카드 낼 때 list.() 쓰면 될듯

In [3]:
import re, random

with open('dict.txt', 'rt', encoding='utf-8') as f:
    s = f.read()

pat = re.compile('^[ㄱ-ㅎ가-힣]+$')
wordDict = dict()
hanbangSet = set()

# 한글로만 이루어져있고, 길이가 2 이상인 단어만 추출
for i in sorted([i for i in s.split() if pat.match(i) and len(i) >= 2], key=lambda x:-len(x)):
    if i[0] not in wordDict:
        wordDict[i[0]] = set()
    wordDict[i[0]].add(i)

# 한방단어 제거 & 추출
delList = list()
for i in wordDict:
    for j in wordDict[i]:
        if j[-1] not in wordDict:
            delList.append(j)
for j in delList:
    hanbangSet.add(j)
    wordDict[j[0]].remove(j)

print('\n 초고수 끝말잇기 ver.0.1')
print(' Ctrl+Z를 입력하면 기권할 수 있습니다.')
round, win, lose = 0, 0, 0

while True:
    # 라운드 시작
    round += 1
    print("\n" + "-" * 50)
    print("\n %d라운드를 시작합니다. 현재 %d승 %d패" % (round, win, lose))
    lastWord = ''
    alreadySet = set()
    firstTurn = True
    resetRound = False

    while True:
        # CPU 턴
        print()
        if firstTurn:
            lastWord = random.choice(list(wordDict[random.choice(list(wordDict.keys()))]))
            alreadySet.add(lastWord)
            print(' CPU : ' + lastWord)
            firstTurn = False
        else:
            firstLetter = lastWord[-1]
            if not list(filter(lambda x: x not in alreadySet, wordDict.get(firstLetter, set()))):
                # 라운드 종료
                print(' CPU : ^Z')
                print('\n [결과] CPU가 기권했습니다. 당신의 승리입니다!')
                win += 1
                break
            else:
                nextWords = sorted(filter(lambda x: x not in alreadySet, wordDict[firstLetter]), key=lambda x:-len(x))[:random.randint(20, 50)]
                lastWord = nextWords[random.randint(0, random.randrange(0, len(nextWords)))]
                alreadySet.add(lastWord)
                print(' CPU : ' + lastWord)

        # 유저 턴
        while True:
            print()
            try:
                yourWord = input(' YOU : ')
            except:
                print('\n [결과] 당신은 기권했습니다. CPU의 승리입니다!')
                print(' [힌트] ', end='')
                print(', '.join(list(filter(lambda x: x not in alreadySet, wordDict.get(lastWord[-1], set())))[:3]))
                resetRound = True
                lose += 1
                break
            firstLetter = yourWord[0]
            if firstLetter != lastWord[-1]:
                print(" [오류] '" + lastWord[-1] + "' (으)로 시작하는 단어를 입력하세요.")
            elif yourWord in hanbangSet:
                print(' [오류] 한방단어는 사용할 수 없습니다.')
            elif yourWord in alreadySet:
                print(' [오류] 이미 나온 단어입니다.')
            elif yourWord not in wordDict.get(firstLetter, set()):
                print(' [오류] 사전에 없는 단어입니다.')
            else:
                alreadySet.add(yourWord)
                lastWord = yourWord
                break

        if resetRound:
            # 라운드 종료
            break


 초고수 끝말잇기 ver.0.1
 Ctrl+Z를 입력하면 기권할 수 있습니다.

--------------------------------------------------

 1라운드를 시작합니다. 현재 0승 0패

 CPU : 령의정

 YOU : 정령

 CPU : 령출력원자로

 YOU : 로키

 CPU : 키카데오이데아

 YOU : 아시아

 CPU : 아말감농축폴라로그라프법

 YOU : 법학도

 CPU : 도급로동임금형태

 YOU : 태환

 CPU : 환경미화원

 YOU : 원자로

 CPU : 로맨티시스트

 YOU : 트롤

 CPU : 롤파쇄기

 YOU : 기창

 CPU : 창자막힘증

 YOU : 증축

 CPU : 축산작업반

 YOU : 반추

 CPU : 추종전동장치

 YOU : 치골

 CPU : 골목골목이

 YOU : 이재용
 [오류] 사전에 없는 단어입니다.

 YOU : 이빨

 CPU : 빨긋빨긋이

 YOU : 이빨
 [오류] 이미 나온 단어입니다.

 YOU : 이명

 CPU : 명충살이고치벌

 YOU : 벌레

 CPU : 레조디칼라브리아

 YOU : 아밀라아제

 CPU : 제비꼬리촉무음

 YOU : 음계

 CPU : 계미진신풍우록

 YOU : 록키
 [오류] 사전에 없는 단어입니다.

 YOU : 록음악
 [오류] 사전에 없는 단어입니다.

 YOU : 록
 [오류] 사전에 없는 단어입니다.

 YOU : 록록
 [오류] 사전에 없는 단어입니다.

 YOU : 녹녹
 [오류] '록' (으)로 시작하는 단어를 입력하세요.

 YOU : 록비

 CPU : 비프로레타리아

 YOU : 아비

 CPU : 비에니아프스키

 YOU : 키비

 CPU : 비자루바다지렁이

 YOU : 


IndexError: string index out of range